In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys

import anndata
import numpy as np
import pandas as pd

from sklearn.decomposition import PCA

# sys.path.append("/home/phil/mixture_embeddings/notebooks/pac")
sys.path.append("/home/phil/mixture_embeddings")
# from util import mixture_embedding
from src.embeddings.mixture_embeddings import (
    get_mixture_embeddings,
    convert_geometry_after,
)


INFO: Using numpy backend


In [3]:
# Set pytorch backend

# os.environ["GEOMSTATS_BACKEND"] = "pytorch"
os.environ["GEOMSTATS_BACKEND"] = "numpy"


In [4]:
# Load data

# adata = anndata.read_h5ad("data/big_table.h5ad")
# adata = anndata.read_h5ad("data/big_table.h5ad")
adata = anndata.read_h5ad("data/americangut_embeddings.h5ad")
adata.X /= adata.X.sum(axis=1)  # normalize

adata_df = adata.to_df()

# Truncate

# adata.X = adata.X.tocsr()
# adata = adata[:1000]

print(adata)
print(adata.to_df().iloc[:10].sum(axis=1))  # check that the data is normalized


AnnData object with n_obs × n_vars = 32608 × 37215
    obs: 'acid_reflux', 'acne_medication', 'acne_medication_otc', 'add_adhd', 'age_cat', 'age_years', 'alcohol_consumption', 'alcohol_frequency', 'alcohol_types_beercider', 'alcohol_types_red_wine', 'alcohol_types_sour_beers', 'alcohol_types_spiritshard_alcohol', 'alcohol_types_unspecified', 'alcohol_types_white_wine', 'allergic_to_i_have_no_food_allergies_that_i_know_of', 'allergic_to_peanuts', 'allergic_to_shellfish', 'allergic_to_tree_nuts', 'allergic_to_unspecified', 'alzheimers', 'animal_age', 'animal_free_text', 'animal_gender', 'animal_origin', 'animal_type', 'animal_type_free_text', 'anonymized_name', 'antibiotic_history', 'appendix_removed', 'artificial_gi_disorder_types_constipation', 'artificial_gi_disorder_types_diarrhea', 'artificial_gi_disorder_types_soft_stools', 'artificial_gi_disorder_types_stomachache', 'artificial_gi_disorder_types_unspecified', 'artificial_gi_disorders', 'artificial_sweeteners', 'artificial_sweetene

In [5]:
# Functions for embeddings

# Euclidean mixture embedding:
def get_euc_mix(ndim):
    otu_embeddings = pd.read_csv(
        f"/data/phil/otu_embeddings/embeddings_euclidean_{ndim}.csv",
        dtype={0: str},
    )
    otu_embeddings = otu_embeddings.set_index(otu_embeddings.columns[0])

    return get_mixture_embeddings(
        otu_table_df=adata_df,
        otu_embeddings_df=otu_embeddings.loc[adata_df.columns],
        space="euclidean",
        embedding_size=ndim,
        return_percent_converged=False,
        fmean_model=None,
        mode=None,
    )


# PCA:
def get_pca(ndim):
    ndim = np.min([ndim, adata.shape[0]])
    pca = PCA(n_components=ndim)
    return pca.fit_transform(adata.X.toarray())


# Hyperbolic mixture embedding:
def get_hyp_mix(ndim):
    otu_embeddings = pd.read_csv(
        f"/data/phil/otu_embeddings/embeddings_hyperbolic_{ndim}.csv",
        dtype={0: str},
    )
    otu_embeddings = otu_embeddings.set_index(otu_embeddings.columns[0])

    return get_mixture_embeddings(
        otu_table_df=adata_df,
        otu_embeddings_df=otu_embeddings.loc[adata_df.columns],
        embedding_size=ndim,
        space="hyperbolic",
        fmean_model="hyperboloid",
        mode="manual",
        return_percent_converged=False,
        convert_back=False,
        n_jobs=-1,
    )


In [25]:
# for ndim in [2, 4, 8, 16, 32, 64, 128]:
# for ndim in [64, 128]:
for ndim in [2]:
    print(ndim)
    # print("\teuc")
    # adata.obsm[f"euc_mix_{ndim}"] = get_euc_mix(ndim)
    # print("\tpca")
    # adata.obsm[f"pca_{ndim}"] = get_pca(ndim)
    # print("\thyp")
    hyp_me = get_hyp_mix(ndim)
    adata.obsm[f"hyp_mix_{ndim}"] = hyp_me
    adata.obsm[f"poi_mix_{ndim}"] = np.array([
        convert_geometry_after("hyperboloid", "poincare", x) for x in hyp_me.values
    ])


2


  0%|          | 12/32608 [00:00<20:58, 25.91it/s]INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend
INFO: Using numpy backend


In [24]:
np.array([
        convert_geometry_after("hyperboloid", "poincare", x) for x in hyp_me.values
    ])


array([[-0.83642154,  0.54526037],
       [-0.83642015,  0.5452597 ],
       [-0.83641783,  0.54525875],
       ...,
       [-0.83644775,  0.54527655],
       [-0.83661565,  0.54541006],
       [-0.836497  ,  0.54531285]])

In [21]:
hyp_me

,0,1,2
Sample,,,
10317.X00179385,-541.350676,352.904673,646.222302
10317.X00179004,-540.813092,352.554374,645.580659
10317.X00179030,-539.953431,351.994327,644.554663
10317.X00179108,-534.287511,348.300642,637.791096
10317.X00179203,-547.397319,356.846750,653.440455
...,...,...,...
10317.BLANK226.6C,-541.316613,352.882273,646.181533
10317.BLANK227.7A,-531.474640,346.467172,634.433443
10317.BLANK227.7D,-552.356676,360.078848,659.360049


In [12]:
convert_geometry_after(
    "hyperboloid", "poincare", hyp_me.values
)

array([[0.94218253, 0.9380399 , 0.93917709],
       [0.9412469 , 0.93710878, 0.93824457],
       [0.93975072, 0.93562015, 0.93675345],
       ...,
       [0.92499398, 0.92092867, 0.92204393],
       [0.96133769, 0.95710924, 0.95827063],
       [1.11914064, 1.11426723, 1.11558446]])

In [19]:
from src.embeddings.frechet_mean_manual import to_poincare_ball_point

to_poincare_ball_point(hyp_me.values[0])

array([-0.83642154,  0.54526037])

In [ ]:
def to_poincare_ball_point(hyperboloid_pt):
    """
    Project the point of the hyperboloid onto the Poincaré ball.
    Post: len(result) == len(hyperboloid_pt) - 1
    
    The last coordinate is the special coordinate
    """
    N = len(hyperboloid_pt) - 1
    return hyperboloid_pt[:N] / (hyperboloid_pt[N] + 1)


In [18]:
hyp_me[0,:2]

KeyError: (0, slice(None, 2, None))

In [ ]:
adata.X = adata.X.tocsr()
for embedding in adata.obsm.keys():
    if type(adata.obsm[embedding]) == pd.DataFrame:
        adata.obsm[embedding] = adata.obsm[embedding].values

adata.write_h5ad("data/big_table_with_embeddings_upto_128.h5ad")


In [ ]:
adata.obsm.keys()

KeysView(AxisArrays with keys: euc_mix_2, pca_2, hyp_mix_2, poi_mix_2, euc_mix_4, pca_4, hyp_mix_4, poi_mix_4, euc_mix_8, pca_8, hyp_mix_8, poi_mix_8, euc_mix_16, pca_16, hyp_mix_16, poi_mix_16, euc_mix_32, pca_32, hyp_mix_32, poi_mix_32, euc_mix_64, pca_64, hyp_mix_64, poi_mix_64, euc_mix_128, pca_128, hyp_mix_128, poi_mix_128)